In [1]:
# success
import os
import cv2
import numpy as np
from tqdm import tqdm

In [2]:
piece_scale = 50 # need to be the factor of 250
concat_scale = 25

original_width = 4000
original_height = 2250

image_folder = './japan/'
target_image_dir = './target/20230824_115144.jpg'

In [3]:
target_image = cv2.imread(target_image_dir)
print(target_image.shape)
target_image = target_image[1:2251, 0:4000]

(2252, 4000, 3)


In [4]:
# np.savez('images_list_nome_80_45.npz', *resize_image_list)
loaded_data = np.load('images_list_nome_80_45.npz')
loaded_images = [loaded_data[key] for key in loaded_data]

In [28]:
resize_height = int(original_height / piece_scale)
resize_width = int(original_width / piece_scale)

bgr_mean_list = []
for i in tqdm(loaded_images):
    bgr_mean_list.append(np.mean(i, axis=(1,0)))
print(len(bgr_mean_list))

replace = np.zeros((original_height, original_width, 3), np.uint8)

selected_image = []

for y in tqdm(range(0, original_height, resize_height)):
    for x in range(0, original_width, resize_width):
        part = target_image[y:y+resize_height, x:x+resize_width]
        bgr = np.mean(part, axis=(1,0))
        min_distance = 195075
        for i in range(len(loaded_images)):
            j = loaded_images[i]
            k = bgr_mean_list[i]
            distance = ((bgr[0]-k[0])**2+
                        (bgr[1]-k[1])**2+
                        (bgr[2]-k[2])**2)
            if distance<min_distance:
                min_distance = distance
                choice = j
                index = i 
        selected_image.append(index) 
        #replace[y:y+resize_height, x:x+resize_width] = choice

100%|██████████| 5213/5213 [00:00<00:00, 22306.78it/s]


5213


100%|██████████| 50/50 [00:10<00:00,  4.81it/s]


In [40]:
def concat_tile_image(files, image_list, n_row=10, n_col=10, WIDTH = 4000, HEIGHT=2250, scale=1.0):
    height = int(HEIGHT/scale)
    width = int(WIDTH/scale)
    concat_list = []

    for idx, image_index in enumerate(image_list):
        # print(idx)
        image = files[image_index]
        image = cv2.imread(image_folder+image)
        # print(image.shape)
        image = cv2.resize(image[1:2251, 0:4000], (width, height), interpolation=cv2.INTER_AREA)
        # print(image.shape)

        if idx % n_col == 0:
            row_list = [image]
        else:
            row_list = row_list + [image]
            if idx % n_col == (n_col -1):
                row_img = np.hstack(row_list)
                # cv2.imshow('a', row_img)
                # cv2.waitKey(0)
                # cv2.destroyAllWindows()
                concat_list = concat_list +[row_img]
        if idx == len(image_list) - 1:
            if idx % n_col < n_col -1:
                n_blank = n_col - (len(image_list) % n_col)
                row_list = row_list + [np.zeros( (width, height, 3) )] * n_blank
                row_img = np.hstack(row_list)
                concat_list = concat_list + [row_img]            
        if idx >= n_row * n_col:
            break
    concat_img = np.vstack(concat_list)
    cv2.imwrite('replace_6.jpg', concat_img)  

In [41]:
files = os.listdir(image_folder) 
concat_tile_image(files, selected_image, 50, 50, original_width, original_height, 25)
# concat_tile_image(mosaic_list_sorted, n_col=n_col, n_row=n_row, scale=piece_scale)